# Notebook d'importation des données de Wikidata

Déroulement de l'exécution du notebook:
1. Récupérer les personnes avec leur parti
2. Nettoyer les données (eg. annee_naissance doit être un integer et pas une chaîne de caractères)
3. Construire la liste des partis à partir de la liste des personnes
4. Créer la table des partis à partir du dataframe des partis:
   - colonnes: id, code_parti, parti_uri
5. On sauve les partis dans une table `wdt_partis`
4. Créer la table des personne avec la contrainte fk_parti_id
   ```sql
   CREATE TABLE wdt_personnes ( 
    index INTEGER PRIMARY KEY, 
    person_uri TEXT,
    nom_prenom TEXT,
    genre_uri TEXT,
    genre_code TEXT,
    genre_label TEXT,
    annee_naissance INTEGER,
    FOREIGN KEY (fk_parti_index) REFERENCES wdt_partis(index)
   );
   ```
6. On insert les personnes avec la méthode `to_sql` du dataframe mais **avec l'option `if_exists='append'`**.

# import des librairies

In [1]:
### Librairie à installer dans l'environnement conda
# qui sera choisi pour exécuter le carnet
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Librairies déjà installées avec Python
import pprint
import csv
import sys

import sqlite3 as sql

import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile

### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
# sys.path.insert(0, '../..')

import sparql_functions as spqf

print("Les imports se sont bien passés")

Les imports se sont bien passés


# étape 1 - import des personnes dans un dataframe

In [2]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [3]:
query = """
SELECT DISTINCT ?item  ?itemLabel  ?gender ?birthYear ?partiUri
        WHERE {
            {
         
          {?item wdt:P39 wd:Q18510612} #p39 position held et Q18510612 pour conseil national
            UNION
            {?item wdt:P17 wd:Q39} # p17 country et Q39 switzerland
         
              }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate;
                wdt:P21 ?gender;
                wdt:P102 ?partiUri;
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?birthYear)
        FILTER(xsd:integer(?birthYear) > 1900 )

          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        } 
     ORDER BY ?birthYear
"""

print ("u nail it")

u nail it


In [4]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

print ("tt est bon")

tt est bon


In [5]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

1147


[['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q672926'],
 ['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q684909'],
 ['http://www.wikidata.org/entity/Q120229',
  'Robert Tobler',
  'http://www.wikidata.org/entity/Q6581097',
  '1901',
  'http://www.wikidata.org/entity/Q13142801']]

In [6]:
### Créer un DataFrame à partir du résultat
df_personnes = pd.DataFrame(r)
df_personnes.columns = ['personne_uri', 'nom_prenom', 'genre_uri', 'annee_naissance', 'parti_uri']
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri
0,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q672926
1,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q684909
2,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q13142801
3,http://www.wikidata.org/entity/Q121118,Gabriel Despland,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q13850128
4,http://www.wikidata.org/entity/Q122856,Joseph Ackermann,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q667725


In [7]:
print(df_personnes.dtypes)

personne_uri       object
nom_prenom         object
genre_uri          object
annee_naissance    object
parti_uri          object
dtype: object


# étape 2 - nettoyage des données

## transformer les années de naissance de str à int

In [8]:
def annee_naissance_int(annee_naissance):
    return int(annee_naissance)

### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['annee_naissance'] = df_personnes.apply(lambda x: annee_naissance_int(x['annee_naissance']), axis=1)

In [9]:
print(df_personnes.dtypes)

personne_uri       object
nom_prenom         object
genre_uri          object
annee_naissance     int64
parti_uri          object
dtype: object


## création du genCode

In [10]:
 def code_genre(genre_uri):
    if genre_uri == 'http://www.wikidata.org/entity/Q6581097':
        # genre masculin
        a = 'M'
    elif genre_uri =='http://www.wikidata.org/entity/Q6581072':
        # genre féminin
        a = 'F'
    else :
        return 'A'
    return a  
     
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['genre_code'] = df_personnes.apply(lambda x: code_genre(x['genre_uri']), axis=1)

In [11]:
### On inspecte les cinq premières lignes qui ont la valeur 'F' = féminin
df_personnes[df_personnes['genre_code']=='F'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,genre_code
105,http://www.wikidata.org/entity/Q73909077,Liselotte Spreng,http://www.wikidata.org/entity/Q6581072,1912,http://www.wikidata.org/entity/Q13850128,F
109,http://www.wikidata.org/entity/Q78072982,Hanny Thalmann,http://www.wikidata.org/entity/Q6581072,1912,http://www.wikidata.org/entity/Q659461,F
120,http://www.wikidata.org/entity/Q1515691,Gertrud Spiess,http://www.wikidata.org/entity/Q6581072,1914,http://www.wikidata.org/entity/Q659461,F
135,http://www.wikidata.org/entity/Q1903085,Martha Ribi,http://www.wikidata.org/entity/Q6581072,1915,http://www.wikidata.org/entity/Q13850128,F
199,http://www.wikidata.org/entity/Q1330086,Elisabeth Lardelli,http://www.wikidata.org/entity/Q6581072,1921,http://www.wikidata.org/entity/Q385258,F


In [12]:
print(df_personnes.dtypes)

personne_uri       object
nom_prenom         object
genre_uri          object
annee_naissance     int64
parti_uri          object
genre_code         object
dtype: object


## création du code_parti

In [13]:
def code_parti(parti_uri):
    if parti_uri == 'http://www.wikidata.org/entity/Q303745':
        # PS
        a = 'PS'
    elif parti_uri =='http://www.wikidata.org/entity/Q385258':
        # UDC
        a = 'UDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q13850128':
        # PRD
        a = 'PRD'
    elif parti_uri =='http://www.wikidata.org/entity/Q659461':
        # PDC
        a = 'PRD'
    elif parti_uri =='http://www.wikidata.org/entity/Q659739':
        # Les Verts
        a = 'Verts'
    elif parti_uri =='http://www.wikidata.org/entity/Q202638':
        # PLR
        a = 'PLR'
    elif parti_uri =='http://www.wikidata.org/entity/Q667785':
        # PLS
        a = 'PLS'
    elif parti_uri =='http://www.wikidata.org/entity/Q550295':
        # alliance des indépendants
        a = 'ALL_IND'
    elif parti_uri =='http://www.wikidata.org/entity/Q545900':
        # PVL
        a = 'PVL'
    elif parti_uri =='http://www.wikidata.org/entity/Q100934025':
        # le centre
        a = 'LC'
    elif parti_uri =='http://www.wikidata.org/entity/Q151768':
        # parti bourgeois démocrate
        a = 'PBD'
    elif parti_uri =='http://www.wikidata.org/entity/Q477228':
        # parti des paysans et artisans indépendants
        a = 'PAI'
    elif parti_uri =='http://www.wikidata.org/entity/Q667718':
        # parti évangélique suisse
        a = 'PEV'
    elif parti_uri =='http://www.wikidata.org/entity/Q661771':
        # parti suisse du travail
        a = 'PST'
    elif parti_uri =='http://www.wikidata.org/entity/Q660046':
        # démocrates suisse ou action nationale
        a = 'DS'
    elif parti_uri =='http://www.wikidata.org/entity/Q668101':
        # ligue des Tessinois
        a = 'lega'
    elif parti_uri =='http://www.wikidata.org/entity/Q2145393':
        # mouvement républicain
        a = 'MR'
    elif parti_uri =='http://www.wikidata.org/entity/Q667836':
        # union démocratique fédérale
        a = 'UDF'
    elif parti_uri =='http://www.wikidata.org/entity/Q681843':
        # parti démocrate suisse
        a = 'PDS'
    elif parti_uri =='http://www.wikidata.org/entity/Q667725':
        # parti chrétien social
        a = 'PDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q663580':
        # mouvement citoyens genevois
        a = 'MCG'
    elif parti_uri =='http://www.wikidata.org/entity/Q362949':
        # organisation progressistes de Suisse
        a = 'POCH'
    elif parti_uri =='http://www.wikidata.org/entity/Q327591':
        # indépendant
        a = 'IND'
    elif parti_uri =='http://www.wikidata.org/entity/Q13124':
        # parti libéral démocrate
        a = 'FDP'
    elif parti_uri =='http://www.wikidata.org/entity/Q684909':
        # collection fédérale Eidgenössische Sammlung
        a = 'CF'
    elif parti_uri =='http://www.wikidata.org/entity/Q677491':
        # mouvement socio-libéral
        a = 'SLB'
    elif parti_uri =='http://www.wikidata.org/entity/Q672946':
        # parti suisse de la liberté
        a = 'PSL'
    elif parti_uri =='http://www.wikidata.org/entity/Q672926':
        # front national
        a = 'FN'
    elif parti_uri =='http://www.wikidata.org/entity/Q3366732':
        # parti socialiste autonome
        a = 'PSA'
    elif parti_uri =='http://www.wikidata.org/entity/Q3366188':
        # parti chrétien-social indépendant
        a = 'PCSI'
    elif parti_uri =='http://www.wikidata.org/entity/Q286683':
        # solidaritéS
        a = 'Sol'
    elif parti_uri =='http://www.wikidata.org/entity/Q255650':
        # la Gauche alternative linke
        a = 'AL'
    elif parti_uri =='http://www.wikidata.org/entity/Q20012603':
        # ensemble à gauche
        a = 'EaG'
    elif parti_uri =='http://www.wikidata.org/entity/Q13142801':
        # Neue Front
        a = 'NeF'
    elif parti_uri =='http://www.wikidata.org/entity/Q1254507':
        # alliance verte et sociale bernoise, grünes bündnis
        a = 'GB'
    elif parti_uri =='http://www.wikidata.org/entity/Q1186111':
        # demokratisch-soziale partei
        a = 'PDC'
    elif parti_uri =='http://www.wikidata.org/entity/Q1084149':
        # parti chrétien-social d'obwald
        a = 'PCS'
    elif parti_uri =='http://www.wikidata.org/entity/Q10594788':
        # action nationale pour le peuple et la patrie
        a = 'NA'
    else:
        return 'A'
        
    return a 

### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['code_parti'] = df_personnes.apply(lambda x: code_parti(x['parti_uri']), axis=1)

In [14]:
### On inspecte les cinq premières lignes qui ont la valeur 'PAI' = xxx
df_personnes[df_personnes['code_parti']=='PAI'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,genre_code,code_parti
5,http://www.wikidata.org/entity/Q125303,Dewet Buri,http://www.wikidata.org/entity/Q6581097,1901,http://www.wikidata.org/entity/Q477228,M,PAI
34,http://www.wikidata.org/entity/Q2543940,Walter Degen,http://www.wikidata.org/entity/Q6581097,1904,http://www.wikidata.org/entity/Q477228,M,PAI
55,http://www.wikidata.org/entity/Q2173417,Rudolf Meier,http://www.wikidata.org/entity/Q6581097,1907,http://www.wikidata.org/entity/Q477228,M,PAI
75,http://www.wikidata.org/entity/Q23888175,Hans Weber,http://www.wikidata.org/entity/Q6581097,1908,http://www.wikidata.org/entity/Q477228,M,PAI
85,http://www.wikidata.org/entity/Q2039809,Otto Locher,http://www.wikidata.org/entity/Q6581097,1910,http://www.wikidata.org/entity/Q477228,M,PAI


# étape 3 - Construire la liste des partis à partir de la liste des personnes

In [15]:
dict_partis = dict()

for index, row in df_personnes.iterrows():
    if row['code_parti'] not in dict_partis:
        dict_partis[row['code_parti']] = row['parti_uri']

df_partis = pd.DataFrame.from_dict({'parti_code': dict_partis.keys(), 'parti_uri': dict_partis.values()})

print(df_partis)

   parti_code                                  parti_uri
0          FN     http://www.wikidata.org/entity/Q672926
1          CF     http://www.wikidata.org/entity/Q684909
2         NeF   http://www.wikidata.org/entity/Q13142801
3         PRD   http://www.wikidata.org/entity/Q13850128
4         PDC     http://www.wikidata.org/entity/Q667725
5         PAI     http://www.wikidata.org/entity/Q477228
6          PS     http://www.wikidata.org/entity/Q303745
7         PLS     http://www.wikidata.org/entity/Q667785
8         PDS     http://www.wikidata.org/entity/Q681843
9     ALL_IND     http://www.wikidata.org/entity/Q550295
10        FDP      http://www.wikidata.org/entity/Q13124
11        PST     http://www.wikidata.org/entity/Q661771
12         DS     http://www.wikidata.org/entity/Q660046
13        UDC     http://www.wikidata.org/entity/Q385258
14        PEV     http://www.wikidata.org/entity/Q667718
15         MR    http://www.wikidata.org/entity/Q2145393
16      Verts     http://www.wi

# étape 4 - Créer la table des partis à partir du dataframe des partis

In [16]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/conseil_national_import.db')
cn

In [17]:
### Tester une requête SQL sur une table existante
cur = cn.cursor()
l = cur.execute("SELECT * FROM person limit 3").fetchall()

In [18]:
[print(e) for e in l]

(1, 'http://www.wikidata.org/entity/Q118295', 'Tornare', 'Manuel', '1951-04-20T00:00:00Z', 'male', 'http://www.wikidata.org/entity/Q6581097')
(2, 'http://www.wikidata.org/entity/Q118327', 'Fässler', 'Hildegard', '1951-06-22T00:00:00Z', 'female', 'http://www.wikidata.org/entity/Q6581072')
(3, 'http://www.wikidata.org/entity/Q118380', 'Frösch', 'Therese', '1951-06-22T00:00:00Z', 'female', 'http://www.wikidata.org/entity/Q6581072')


[None, None, None]

In [19]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_partis.to_sql(name='wdt_partis', con=cn, if_exists='replace', index_label='id_pa')
except Exception as e:
    print('Erreur: ',  e)

Erreur:  Execution failed on sql 'DROP TABLE "wdt_partis"': database is locked


# étape 6 - créer la table des personnes avec la contraintes fk_partis_index

In [20]:
# On supprime la table si elle est déjà là
query = "DROP TABLE IF EXISTS wdt_personnes"
cur = cn.cursor()
cur.execute(query)

OperationalError: database is locked

In [108]:
query = """
CREATE TABLE wdt_personnes ( 
 id INTEGER PRIMARY KEY, 
 personne_uri TEXT,
 nom_prenom TEXT,
 genre_uri TEXT,
 genre_code TEXT,
 annee_naissance INTEGER,
 fk_parti_id INTEGER,
 FOREIGN KEY (fk_parti_id) REFERENCES wdt_partis(id_pa)
 )
 """

cur = cn.cursor()
cur.execute(query)

# étape 7 - insertion des individus dans la table wdt_personnes

In [109]:
def ajout_fk_parti_id(parti_code):
    ligne_parti = df_partis.index[df_partis['parti_code']== parti_code].tolist()[0]
    return ligne_parti

print(ajout_fk_parti_id('UDC'))

13


In [110]:
### On ajoute une nouvelle colonne et y on met pour chaque ligne le résultat de la fonction
df_personnes['fk_parti_id'] = df_personnes.apply(lambda x: ajout_fk_parti_id(x['code_parti']), axis=1)

In [111]:
### On inspecte les cinq premières lignes qui ont la valeur 'PAI' = xxx
df_personnes[df_personnes['code_parti']=='UDC'][:5]

,personne_uri,nom_prenom,genre_uri,annee_naissance,parti_uri,genre_code,code_parti,fk_parti_id
94,http://www.wikidata.org/entity/Q16101728,Otto Bretscher,http://www.wikidata.org/entity/Q6581097,1911,http://www.wikidata.org/entity/Q385258,M,UDC,13
114,http://www.wikidata.org/entity/Q1582102,Hans Roth,http://www.wikidata.org/entity/Q6581097,1913,http://www.wikidata.org/entity/Q385258,M,UDC,13
125,http://www.wikidata.org/entity/Q2543559,Walter Baumann,http://www.wikidata.org/entity/Q6581097,1914,http://www.wikidata.org/entity/Q385258,M,UDC,13
142,http://www.wikidata.org/entity/Q1579071,Hans Conzett,http://www.wikidata.org/entity/Q6581097,1915,http://www.wikidata.org/entity/Q385258,M,UDC,13
151,http://www.wikidata.org/entity/Q74448938,Georg Brosi,http://www.wikidata.org/entity/Q6581097,1916,http://www.wikidata.org/entity/Q385258,M,UDC,13


In [117]:
df_personnes = df_personnes.drop(['code_parti', 'parti_uri'], axis=1)

In [118]:
df_personnes.head()

,personne_uri,nom_prenom,genre_uri,annee_naissance,genre_code,fk_parti_id
0,http://www.wikidata.org/entity/Q1447990,Franz Landolt,http://www.wikidata.org/entity/Q6581097,1901,M,0
1,http://www.wikidata.org/entity/Q122856,Joseph Ackermann,http://www.wikidata.org/entity/Q6581097,1901,M,1
2,http://www.wikidata.org/entity/Q125303,Dewet Buri,http://www.wikidata.org/entity/Q6581097,1901,M,2
3,http://www.wikidata.org/entity/Q1758783,Paul Schib,http://www.wikidata.org/entity/Q6581097,1901,M,3
4,http://www.wikidata.org/entity/Q120229,Robert Tobler,http://www.wikidata.org/entity/Q6581097,1901,M,4


In [119]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_personnes.to_sql(name='wdt_personnes', con=cn, if_exists='append', index_label='id')
except Exception as e:
    print('Erreur: ',  e)